<a href="https://colab.research.google.com/github/MoawwazTahir/Wardrobe_proj/blob/main/qwen2_vl_2b_api_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install flask pyngrok transformers torch Pillow requests qwen_vl_utils auto-gptq optimum



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      S

In [2]:
import os
import torch
import gc
from PIL import Image
from flask import Flask, request, jsonify
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info  # Assuming process_vision_info is available
import requests
from io import BytesIO
from pyngrok import ngrok

# Set up the Ngrok authentication token
ngrok_auth_token = "2p0tNdsPkUnSJELTDbhUEuYaC0I_6GRKZSEK5Y4xJWj3RApdK"  # Replace with your actual Ngrok token
ngrok.set_auth_token(ngrok_auth_token)  # Set Ngrok auth token

# Initialize Flask app
app = Flask(__name__)

# Load the model and processor
def load_model_and_processor(min_pixels=256 * 28 * 28, max_pixels=1280 * 28 * 28):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    try:
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            "Qwen/Qwen2-VL-2B-Instruct-GPTQ-Int4",
            torch_dtype=torch.bfloat16 if device.type == "cuda" else torch.float32,
            device_map="auto",  # You can remove this line if it conflicts with single GPU usage
        )
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None

    try:
        processor = AutoProcessor.from_pretrained(
            "Qwen/Qwen2-VL-2B-Instruct-GPTQ-Int4", min_pixels=min_pixels, max_pixels=max_pixels
        )
        print(f"Processor loaded successfully with image resizing ({min_pixels} - {max_pixels} pixels).")
    except Exception as e:
        print(f"Error loading processor: {e}")
        return None, None, None

    model.to(device)  # Move the model to the specified device
    model.eval()
    return model, processor, device

# Load model and processor
model, processor, device = load_model_and_processor()



Using device: cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Processor loaded successfully with image resizing (200704 - 1003520 pixels).


In [ ]:
def get_generate_method(model):
    return model.generate

def process_image_and_generate_caption(image_url: str):
    try:
        # Fetch image from URL
        if image_url.startswith('http'):
            response = requests.get(image_url)

            # Check if the response is an image
            if 'image' not in response.headers.get('Content-Type', ''):
                raise ValueError(f"URL does not return a valid image: {image_url}")

            img = Image.open(BytesIO(response.content)).convert("RGB")
            print(f"Image fetched from URL: {image_url}")
        else:
            img = Image.open(image_url).convert("RGB")
            print(f"Image loaded from file: {image_url}")

        # Resize image
        img = img.resize((1024, 1024))
        print(f"Image size after resizing: {img.size}")

        # Prepare image data using process_vision_info (to match the original code's intent)
        messages = [{
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": "Describe this image."}
            ]
        }]

        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        # Process vision info (this should match how your original code processed images)
        image_inputs, video_inputs = process_vision_info(messages)

        # Prepare the input for the model using the processor
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        ).to(device)

        # Check the structure of the inputs
        print(f"Inputs from processor: {inputs}")

        # Ensure inputs are valid
        if 'input_ids' not in inputs or 'attention_mask' not in inputs or 'pixel_values' not in inputs:
            raise ValueError("Invalid inputs returned by the processor.")

        # Generate caption
        generate = get_generate_method(model)
        with torch.no_grad():
            generated_ids = generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )

        # Return the caption
        return output_text[0], img

    except Exception as e:
        print(f"Error processing image: {e}")
        return None, None

@app.route('/generate_caption', methods=['POST'])
def generate_caption():
    data = request.get_json()

    # Ensure images are provided in the form of a list of URLs or file paths
    if "images" not in data:
        return jsonify({"error": "No images provided"}), 400

    images = data["images"]

    # Process each image in bulk and return results
    results = []
    for image_url in images:
        caption, image = process_image_and_generate_caption(image_url)
        if caption:
            results.append({
                "image": image_url,
                "caption": caption
            })
        else:
            results.append({
                "image": image_url,
                "error": "Failed to generate caption"
            })

    return jsonify({"results": results, "status": "success"})


# Run the Flask app with Ngrok
if __name__ == "__main__":
    # Start Ngrok tunnel for Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Flask app is publicly available at {public_url}")

    # Run the Flask app
    app.run(debug=False)


Flask app is publicly available at NgrokTunnel: "https://22f1-34-169-15-186.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Error processing image: URL does not return a valid image: https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Football_in_Bloomington%2C_Indiana%2C_1995.jpg/190px-Football_in_Bloomington%2C_Indiana%2C_1995.jpg
Image fetched from URL: https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Scotland_v_Italy_2013_RLWC_%283%29.jpg/192px-Scotland_v_Italy_2013_RLWC_%283%29.jpg
Image size after resizing: (1024, 1024)
Inputs from processor: {'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[-0.1864, -0.1864, -0.1864,  ..., -0.6270, -0.6412, -0.6412],
        [-0.0842, -0.0842, -0.0842,  ..., -0.6981, -0.6697, -0.6555],
        [-0.1572, -0.1572, -0.1572,  ..., -0.6839, -0.6839, -0.6981],
        ...,
        [-0.5222, -0.5660, -0.5952,  ..., -0.4706, -0.4848, -0.4990],
        [ 0.3537,  0.2515,  0.1347,  ..., -0.4564, -0.4848, -0.5275],


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 10:44:29] "POST /generate_caption HTTP/1.1" 200 -


Error processing image: URL does not return a valid image: https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Football_in_Bloomington%2C_Indiana%2C_1995.jpg/190px-Football_in_Bloomington%2C_Indiana%2C_1995.jpg
Image fetched from URL: https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Scotland_v_Italy_2013_RLWC_%283%29.jpg/192px-Scotland_v_Italy_2013_RLWC_%283%29.jpg
Image size after resizing: (1024, 1024)
Inputs from processor: {'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[-0.1864, -0.1864, -0.1864,  ..., -0.6270, -0.6412, -0.6412],
        [-0.0842, -0.0842, -0.0842,  ..., -0.6981, -0.6697, -0.6555],
        [-0.1572, -0.1572, -0.1572,  ..., -0.6839, -0.6839, -0.6981],
        ...,
        [-0.5222, -0.5660, -0.5952,  ..., -0.4706, -0.4848, -0.4990],
        [ 0.3537,  0.2515,  0.1347,  ..., -0.4564, -0.4848, -0.5275],


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 10:45:07] "POST /generate_caption HTTP/1.1" 200 -


Image fetched from URL: https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Aidan_O%27Mahony_%26_Eoin_Bradley.jpg/189px-Aidan_O%27Mahony_%26_Eoin_Bradley.jpg
Image size after resizing: (1024, 1024)
Inputs from processor: {'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[ 0.9084,  0.9230,  0.9814,  ...,  0.3542,  0.1551, -0.0867],
        [-0.0696, -0.5806, -1.0623,  ...,  0.1551,  0.4110,  0.6528],
        [-0.0842, -0.0696, -0.0259,  ..., -0.5986, -0.5844, -0.5986],
        ...,
        [ 0.2661,  0.2661,  0.2661,  ..., -0.9541, -0.9825, -0.9825],
        [ 0.2661,  0.2661,  0.2661,  ..., -0.8688, -0.9114, -0.9399],
        [ 0.2953,  0.2953,  0.2953,  ..., -0.8830, -0.8830, -0.8830]],
       device='cuda:0'), 'image_grid_thw': tensor([[ 1, 70, 70]], device='cuda:0')}
Image fetched from URL: https://upload.wikimedia.org/wikipedia/c

INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 10:46:16] "POST /generate_caption HTTP/1.1" 200 -
